Based on 
http://www.jasq.org/just-another-scala-quant/new-agey-interviews-at-the-grocery-startup


You walk into a grocery store with a grocery bag and some cash, to buy groceries for a week.
 
1. your bag can hold ten pounds.
2. You have $100
3. You need about 2000 calories a day, so a weekly shopping trip is about 14,000 calories.
4. You must purchase at least 4 ounces of each grocery item.



In [8]:
from ortools.linear_solver import pywraplp

In [9]:
maxWeight = 10
maxCost = 100
minCals = 14000
minShop = 4/16.0 #16 ounces per pound

Dataset
---

**Calories Per Pound**

Ham, 650 cals

Lettuce, 70 cals

Cheese, 1670 cals

Tuna, 830 cals

Bread, 1300 cals


**Price Per Pound**

Ham, $4

Lettuce, $1.5

Cheese, $5

Tuna, $20

Bread, $1.20


In [10]:
#[name, calories, prices]
food =  [['ham',650, 4],
		['lettuce',70,1.5],
		['cheese',1670,5],
		['tuna',830,20],
		['bread',1300,1.20]]

Create the solver and some variables

In [11]:
solver = pywraplp.Solver('SolveSimpleSystem',pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)

variable_list = [[]] * len(food)
for i in range(0, len(food)):
    #you must buy at least minShop of each
    variable_list[i] = solver.NumVar(minShop, solver.infinity(), food[i][0])
    

Define the constraints

In [12]:
constraint_list=[]
#Constraint 1: totalWeight<maxWeight
#>=0 ham + lettuce + cheese + tuna + bread <= maxWeight
constraint_list.append(solver.Constraint(0, maxWeight))
for i in range(0, len(food)):
    constraint_list[0].SetCoefficient(variable_list[i],1)

#Constraint 2: total Price<=maxCost
constraint_list.append(solver.Constraint(0, maxCost))
for i in range(0, len(food)):
    constraint_list[1].SetCoefficient(variable_list[i],food[i][2])

#Constraint 3: total Calories>=minCals
constraint_list.append(solver.Constraint(minCals, minCals + 100))
for i in range(0, len(food)):
    constraint_list[2].SetCoefficient(variable_list[i],food[i][1])
    

Define the objective functions

In [13]:
# Define our objective: minimizing cost
objective = solver.Objective()
for i in range(0, len(food)):
    objective.SetCoefficient(variable_list[i], food[i][2])
objective.SetMinimization()


Solve!

In [14]:
result_status = solver.Solve()

if result_status == solver.OPTIMAL:
		print('Successful solve.')
		# The problem has an optimal solution.
		print(('Problem solved in %f milliseconds' % solver.wall_time()))
		# The objective value of the solution.
		print(('Optimal objective value = %f' % solver.Objective().Value()))
		# The value of each variable in the solution.
		for variable in variable_list:
			print(('%s = %f' % (variable.name(), variable.solution_value())))
elif solve_status == assignment.INFEASIBLE:
    print('No solution found.')

Successful solve.
Problem solved in 43.000000 milliseconds
Optimal objective value = 33.779054
ham = 0.250000
lettuce = 0.250000
cheese = 4.290541
tuna = 0.250000
bread = 4.959459
